In [2]:
#!/usr/bin/env python3

import pandas as pd
import errno    
import os
import numpy as np
import math

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

'''
Author: Enrico Ceccolini
    TODO write the description
'''

#datadir = "/datasets/eurora_data/db/"
datadir = "/datasets/eurora_data/db1/"

# settings
interval_comment = "WholeData"
suffix = "_1min_"
# settings Andrea
andrea_start_time = pd.to_datetime('2014-03-31')
andrea_end_time = pd.to_datetime('2014-05-01')
# settings Alina
alina_start_time = pd.to_datetime('2014-09-30')
alina_end_time = pd.to_datetime('2014-11-01')

infile_large_jobs = datadir + "CPUs/" + interval_comment + "/large_jobs_real_pow_final.csv"
interval_large_jobs = pd.read_csv(infile_large_jobs, index_col=0)

nodes 1-16 and 25-32 have a maximum frequency of 2.1GHz (CPUs-2100), nodes 17-24 have a maximum frequency of 2.2GHz (CPUs-2200), and nodes 33-64 have a maximum frequency of 3.1GHz (CPUs-3100). For the accelerators, nodes from 0 to 32 embed Intel Xeon Phi accelerators, whereas nodes from 33 to 64 embed Nvidia Kepler GPUs.

In [3]:
interval_large_jobs.head(1)

,job_id_string,queue,start_time,run_start_time,end_time,user,node_req,cpu_req,mem_req,time_req,exit_status,real_pow,runned_alone,real_pow_quality,n_2_1,n_2_2,n_3_1,gpu_req,mic_req,job_name
0,498458.node129,parallel,2014-03-31 00:30:02,2014-03-31 00:30:02,2014-03-31 01:22:40,alupi000@node129.eurora.cineca.it,1,16,14,04:00,COMPLETED,0.0,True,0.0,0,1,0,NaN,NaN,rth2e6t0


In [4]:
interval_large_jobs.shape

(64000, 20)

In [5]:
interval_large_jobs = interval_large_jobs[interval_large_jobs['real_pow'] != 0]
interval_large_jobs = interval_large_jobs[interval_large_jobs['real_pow_quality'] != 0]

### Compute the mean core power consumption for the first type of cpu (2.1 GHz) 

In [12]:
grouped_user_data = interval_large_jobs.groupby('job_name')['job_id_string'].nunique().reset_index(name='counts')
print("{} distinct jobs".format(grouped_user_data.shape[0]))
grouped_user_data['count_2_1'] = 0
grouped_user_data['accuracy_2_1'] = 0.0
grouped_user_data['count_2_2'] = 0
grouped_user_data['accuracy_2_2'] = 0.0
grouped_user_data['count_3_1'] = 0
grouped_user_data['accuracy_3_1'] = 0.0

11017 distinct jobs


In [13]:
grouped_user_data['counts'].sum()

57658

In [15]:
grouped_user_data = grouped_user_data[grouped_user_data['counts'] > 1]
print("{} distinct jobs".format(grouped_user_data.shape[0]))

3723 distinct jobs


In [16]:
i = 0
for node_index, node_row in grouped_user_data.iterrows():
    user_jobs = interval_large_jobs[interval_large_jobs['job_name'] == node_row['job_name']]
    user_jobs_2_1 = user_jobs[user_jobs['node_req'] == user_jobs['n_2_1']]
    user_jobs_2_2 = user_jobs[user_jobs['node_req'] == user_jobs['n_2_2']]
    user_jobs_3_1 = user_jobs[user_jobs['node_req'] == user_jobs['n_3_1']]
    
    mean_core_2_1 = user_jobs_2_1['real_pow'].sum() / user_jobs_2_1['cpu_req'].sum()
    mean_core_2_2 = user_jobs_2_2['real_pow'].sum() / user_jobs_2_2['cpu_req'].sum()
    mean_core_3_1 = user_jobs_3_1['real_pow'].sum() / user_jobs_3_1['cpu_req'].sum()
    
    user_jobs_2_1['predicted'] = mean_core_2_1 * user_jobs_2_1['cpu_req']
    user_jobs_2_2['predicted'] = mean_core_2_2 * user_jobs_2_2['cpu_req']
    user_jobs_3_1['predicted'] = mean_core_3_1 * user_jobs_3_1['cpu_req']
    
    if(user_jobs_2_1.shape[0] != 0):
        grouped_user_data.loc[node_index, 'accuracy_2_1'] = r2_score(user_jobs_2_1['real_pow'], user_jobs_2_1['predicted'])
        grouped_user_data.loc[node_index, 'count_2_1'] = user_jobs_2_1.shape[0]
    if(user_jobs_2_2.shape[0] != 0):
        grouped_user_data.loc[node_index, 'accuracy_2_2'] = r2_score(user_jobs_2_2['real_pow'], user_jobs_2_2['predicted'])
        grouped_user_data.loc[node_index, 'count_2_2'] = user_jobs_2_2.shape[0]
    if(user_jobs_3_1.shape[0] != 0):
        grouped_user_data.loc[node_index, 'accuracy_3_1'] = r2_score(user_jobs_3_1['real_pow'], user_jobs_3_1['predicted'])
        grouped_user_data.loc[node_index, 'count_3_1'] = user_jobs_3_1.shape[0]
        
    #if(user_jobs_2_1.shape[0] != 0):
    #    grouped_user_data.loc[node_index, 'accuracy_2_1'] = mean_squared_error(user_jobs_2_1['real_pow'], user_jobs_2_1['predicted'])
    #    grouped_user_data.loc[node_index, 'count_2_1'] = user_jobs_2_1.shape[0]
    #if(user_jobs_2_2.shape[0] != 0):
    #    grouped_user_data.loc[node_index, 'accuracy_2_2'] = mean_squared_error(user_jobs_2_2['real_pow'], user_jobs_2_2['predicted'])
    #    grouped_user_data.loc[node_index, 'count_2_2'] = user_jobs_2_2.shape[0]
    #if(user_jobs_3_1.shape[0] != 0):
    #    grouped_user_data.loc[node_index, 'accuracy_3_1'] = mean_squared_error(user_jobs_3_1['real_pow'], user_jobs_3_1['predicted'])
    #    grouped_user_data.loc[node_index, 'count_3_1'] = user_jobs_3_1.shape[0]
    
    i += 1
    print(i)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


13.597731225351458
nan
nan
1
13.63953623889611
nan
nan
2
13.70175664205153
nan
29.045169454239062
3
nan
nan
28.96941436305222


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-c

4
8.72986636747836
nan
nan
5
nan
nan
nan
6
nan
nan
28.4247901540101
7
12.868554332614892
12.771439056092515
nan
8
12.767033761987188
12.6824614215775
nan
9
12.709653366919268
12.938677016627782
nan
10
13.53493957260605
14.477352122103621
28.0734265730211
11
13.303482184650639
nan
nan


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/inde

...

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing

3718
nan
nan
24.46808328575289
3719
10.806611994336906
10.987069004952579
24.89157436211937
3720
nan
nan
25.0123671022567


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:8: RuntimeWarning: invalid value encountered in double_scalars
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.5/dist-packages/ipykernel

3721
28.000768430844097
26.287036498187497
63.85316642687813
3722
nan
nan
31.252571078670872
3723


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/inde

In [18]:
grouped_user_data.sort_values('counts', ascending=False)

,job_name,counts,count_2_1,accuracy_2_1,count_2_2,accuracy_2_2,count_3_1,accuracy_3_1
3398,STDIN,2165,585,0.484154,91,0.435329,1428,-0.315849
4185,cosmomc_nMin20_,1834,982,0.000000,446,0.000000,406,0.000000
4951,job,1233,415,0.949932,83,0.780917,598,0.909337
2671,DL_POLY_multi,1094,129,0.000000,30,0.000000,935,0.000000
6269,renzocomputa,934,305,0.725902,74,0.605861,177,0.358527
6837,run-nscf.ll,808,291,0.610567,178,0.380983,339,0.000000
5412,lamp_PMF_dum,625,0,0.000000,57,0.000000,568,0.000000
5806,opnRNA,619,0,0.000000,0,0.000000,602,-2.962345
9814,run_0,569,70,0.000000,24,0.000000,475,0.000000
4163,convlwalle,543,0,0.000000,60,0.989233,483,0.882531


In [78]:
interval_large_jobs_2_1 = interval_large_jobs[interval_large_jobs['node_req'] == interval_large_jobs['n_2_1']]
print("There are {} jobs completely running on this type of cpu".format(interval_large_jobs_2_1.shape[0]))
mean_core_2_1 = interval_large_jobs_2_1['real_pow'].sum() / interval_large_jobs_2_1['cpu_req'].sum()
print("The average per core is {} W".format(mean_core_2_1))
interval_large_jobs_2_1['predicted'] = mean_core_2_1 * interval_large_jobs_2_1['cpu_req']
print("Accuracy: {}".format(r2_score(interval_large_jobs_2_1['real_pow'], interval_large_jobs_2_1['predicted'])))

### Esclude the jobs with low quality of approximation
quality_large_jobs_2_1 = interval_large_jobs_2_1[interval_large_jobs_2_1['real_pow'] != 0]
quality_large_jobs_2_1 = quality_large_jobs_2_1[quality_large_jobs_2_1['real_pow_quality'] != 0]
print("There are {} jobs completely running on this type of cpu with that quality".format(quality_large_jobs_2_1.shape[0]))
mean_core_2_1 = quality_large_jobs_2_1['real_pow'].sum() / quality_large_jobs_2_1['cpu_req'].sum()
print("The average per core is {} W".format(mean_core_2_1))
quality_large_jobs_2_1['predicted'] = mean_core_2_1 * quality_large_jobs_2_1['cpu_req']
print("Accuracy: {}".format(r2_score(quality_large_jobs_2_1['real_pow'], quality_large_jobs_2_1['predicted'])))

### Esclude jobs that ran concurrently
quality_large_jobs_2_1 = interval_large_jobs_2_1[interval_large_jobs_2_1['real_pow'] != 0]
quality_large_jobs_2_1 = quality_large_jobs_2_1[quality_large_jobs_2_1['real_pow_quality'] != 0]
quality_large_jobs_2_1 = quality_large_jobs_2_1[quality_large_jobs_2_1['runned_alone'] == True]
print("There are {} jobs completely running on this type of cpu with that quality".format(quality_large_jobs_2_1.shape[0]))
mean_core_2_1 = quality_large_jobs_2_1['real_pow'].sum() / quality_large_jobs_2_1['cpu_req'].sum()
print("The average per core is {} W".format(mean_core_2_1))
quality_large_jobs_2_1['predicted'] = mean_core_2_1 * quality_large_jobs_2_1['cpu_req']
print("Accuracy: {}".format(r2_score(quality_large_jobs_2_1['real_pow'], quality_large_jobs_2_1['predicted'])))

There are 13265 jobs completely running on this type of cpu
The average per core is 11.614662777744632 W
Accuracy: 0.604018857666945
There are 12680 jobs completely running on this type of cpu with that quality
The average per core is 12.188997397054441 W
Accuracy: 0.6961530048060831
There are 10089 jobs completely running on this type of cpu with that quality
The average per core is 11.928331731218778 W
Accuracy: 0.6596372349292074


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


### Andrea's interval

In [80]:
andrea_interval_large_jobs =interval_large_jobs[pd.to_datetime(interval_large_jobs['run_start_time']) >= andrea_start_time]
andrea_interval_large_jobs =andrea_interval_large_jobs[pd.to_datetime(andrea_interval_large_jobs['run_start_time']) <= andrea_end_time]

In [81]:
interval_large_jobs_2_1 = andrea_interval_large_jobs[andrea_interval_large_jobs['node_req'] == andrea_interval_large_jobs['n_2_1']]
print("There are {} jobs completely running on this type of cpu".format(interval_large_jobs_2_1.shape[0]))
mean_core_2_1 = interval_large_jobs_2_1['real_pow'].sum() / interval_large_jobs_2_1['cpu_req'].sum()
print("The average per core is {} W".format(mean_core_2_1))
interval_large_jobs_2_1['predicted'] = mean_core_2_1 * interval_large_jobs_2_1['cpu_req']
print("Accuracy: {}".format(r2_score(interval_large_jobs_2_1['real_pow'], interval_large_jobs_2_1['predicted'])))

### Esclude the jobs with low quality of approximation
quality_large_jobs_2_1 = interval_large_jobs_2_1[interval_large_jobs_2_1['real_pow'] != 0]
quality_large_jobs_2_1 = quality_large_jobs_2_1[quality_large_jobs_2_1['real_pow_quality'] != 0]
print("There are {} jobs completely running on this type of cpu with that quality".format(quality_large_jobs_2_1.shape[0]))
mean_core_2_1 = quality_large_jobs_2_1['real_pow'].sum() / quality_large_jobs_2_1['cpu_req'].sum()
print("The average per core is {} W".format(mean_core_2_1))
quality_large_jobs_2_1['predicted'] = mean_core_2_1 * quality_large_jobs_2_1['cpu_req']
print("Accuracy: {}".format(r2_score(quality_large_jobs_2_1['real_pow'], quality_large_jobs_2_1['predicted'])))

### Esclude jobs that ran concurrently
quality_large_jobs_2_1 = interval_large_jobs_2_1[interval_large_jobs_2_1['real_pow'] != 0]
quality_large_jobs_2_1 = quality_large_jobs_2_1[quality_large_jobs_2_1['real_pow_quality'] != 0]
quality_large_jobs_2_1 = quality_large_jobs_2_1[quality_large_jobs_2_1['runned_alone'] == True]
print("There are {} jobs completely running on this type of cpu with that quality".format(quality_large_jobs_2_1.shape[0]))
mean_core_2_1 = quality_large_jobs_2_1['real_pow'].sum() / quality_large_jobs_2_1['cpu_req'].sum()
print("The average per core is {} W".format(mean_core_2_1))
quality_large_jobs_2_1['predicted'] = mean_core_2_1 * quality_large_jobs_2_1['cpu_req']
print("Accuracy: {}".format(r2_score(quality_large_jobs_2_1['real_pow'], quality_large_jobs_2_1['predicted'])))

There are 5064 jobs completely running on this type of cpu
The average per core is 12.143704579416486 W
Accuracy: 0.6683120957773678
There are 4900 jobs completely running on this type of cpu with that quality
The average per core is 12.529485612237833 W
Accuracy: 0.7033161874156395
There are 3450 jobs completely running on this type of cpu with that quality
The average per core is 11.986636720341757 W
Accuracy: 0.6820671373736893


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


### Compute the mean core power consumption for the first type of cpu (2.2 GHz) 

In [85]:
interval_large_jobs_2_2 = interval_large_jobs[interval_large_jobs['node_req'] == interval_large_jobs['n_2_2']]
print("There are {} jobs completely running on this type of cpu".format(interval_large_jobs_2_2.shape[0]))
mean_core_2_2 = interval_large_jobs_2_2['real_pow'].sum() / interval_large_jobs_2_2['cpu_req'].sum()
print("The average per core is {} W".format(mean_core_2_2))
interval_large_jobs_2_2['predicted'] = mean_core_2_2 * interval_large_jobs_2_2['cpu_req']
print("Accuracy: {}".format(r2_score(interval_large_jobs_2_2['real_pow'], interval_large_jobs_2_2['predicted'])))

quality_large_jobs_2_2 = interval_large_jobs_2_2[interval_large_jobs_2_2['real_pow'] != 0]
quality_large_jobs_2_2 = quality_large_jobs_2_2[quality_large_jobs_2_2['real_pow_quality'] != 0]
print("There are {} jobs completely running on this type of cpu with that quality".format(quality_large_jobs_2_2.shape[0]))
mean_core_2_2 = quality_large_jobs_2_2['real_pow'].sum() / quality_large_jobs_2_2['cpu_req'].sum()
print("The average per core is {} W".format(mean_core_2_2))
quality_large_jobs_2_2['predicted'] = mean_core_2_2 * quality_large_jobs_2_2['cpu_req']
print("Accuracy: {}".format(r2_score(quality_large_jobs_2_2['real_pow'], quality_large_jobs_2_2['predicted'])))

There are 4570 jobs completely running on this type of cpu
The average per core is 13.80509054541727 W
Accuracy: 0.44371127080645256
There are 4294 jobs completely running on this type of cpu with that quality
The average per core is 14.664222292746754 W
Accuracy: 0.534215782066189


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


### Andrea's interval

In [86]:
interval_large_jobs_2_2 = andrea_interval_large_jobs[andrea_interval_large_jobs['node_req'] == andrea_interval_large_jobs['n_2_2']]
print("There are {} jobs completely running on this type of cpu".format(interval_large_jobs_2_2.shape[0]))
mean_core_2_2 = interval_large_jobs_2_2['real_pow'].sum() / interval_large_jobs_2_2['cpu_req'].sum()
print("The average per core is {} W".format(mean_core_2_2))
interval_large_jobs_2_2['predicted'] = mean_core_2_2 * interval_large_jobs_2_2['cpu_req']
print("Accuracy: {}".format(r2_score(interval_large_jobs_2_2['real_pow'], interval_large_jobs_2_2['predicted'])))

quality_large_jobs_2_2 = interval_large_jobs_2_2[interval_large_jobs_2_2['real_pow'] != 0]
quality_large_jobs_2_2 = quality_large_jobs_2_2[quality_large_jobs_2_2['real_pow_quality'] != 0]
print("There are {} jobs completely running on this type of cpu with that quality".format(quality_large_jobs_2_2.shape[0]))
mean_core_2_2 = quality_large_jobs_2_2['real_pow'].sum() / quality_large_jobs_2_2['cpu_req'].sum()
print("The average per core is {} W".format(mean_core_2_2))
quality_large_jobs_2_2['predicted'] = mean_core_2_2 * quality_large_jobs_2_2['cpu_req']
print("Accuracy: {}".format(r2_score(quality_large_jobs_2_2['real_pow'], quality_large_jobs_2_2['predicted'])))

There are 1428 jobs completely running on this type of cpu
The average per core is 14.932216042263958 W
Accuracy: 0.5294936382852904
There are 1362 jobs completely running on this type of cpu with that quality
The average per core is 15.45302698203022 W
Accuracy: 0.5504581852733055


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


### Compute the mean core power consumption for the first type of cpu (3.1 GHz) 

In [83]:
interval_large_jobs_3_1 = interval_large_jobs[interval_large_jobs['node_req'] == interval_large_jobs['n_3_1']]
print("There are {} jobs completely running on this type of cpu".format(interval_large_jobs_3_1.shape[0]))
mean_core_3_1 = interval_large_jobs_3_1['real_pow'].sum() / interval_large_jobs_3_1['cpu_req'].sum()
print("The average per core is {} W".format(mean_core_3_1))
interval_large_jobs_3_1['predicted'] = mean_core_3_1 * interval_large_jobs_3_1['cpu_req']
print("Accuracy: {}".format(r2_score(interval_large_jobs_3_1['real_pow'], interval_large_jobs_3_1['predicted'])))

quality_large_jobs_3_1 = interval_large_jobs_3_1[interval_large_jobs_3_1['real_pow'] != 0]
quality_large_jobs_3_1 = quality_large_jobs_3_1[quality_large_jobs_3_1['real_pow_quality'] != 0]
print("There are {} jobs completely running on this type of cpu with that quality".format(quality_large_jobs_3_1.shape[0]))
mean_core_3_1 = quality_large_jobs_3_1['real_pow'].sum() / quality_large_jobs_3_1['cpu_req'].sum()
print("The average per core is {} W".format(mean_core_3_1))
quality_large_jobs_3_1['predicted'] = mean_core_3_1 * quality_large_jobs_3_1['cpu_req']
print("Accuracy: {}".format(r2_score(quality_large_jobs_3_1['real_pow'], quality_large_jobs_3_1['predicted'])))

There are 43708 jobs completely running on this type of cpu
The average per core is 24.842150667100178 W
Accuracy: 0.5806440166370469
There are 38419 jobs completely running on this type of cpu with that quality
The average per core is 27.562140558509466 W
Accuracy: 0.6072288411283724


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


### Andrea's interval

In [87]:
interval_large_jobs_3_1 = andrea_interval_large_jobs[andrea_interval_large_jobs['node_req'] == andrea_interval_large_jobs['n_3_1']]
print("There are {} jobs completely running on this type of cpu".format(interval_large_jobs_3_1.shape[0]))
mean_core_3_1 = interval_large_jobs_3_1['real_pow'].sum() / interval_large_jobs_3_1['cpu_req'].sum()
print("The average per core is {} W".format(mean_core_3_1))
interval_large_jobs_3_1['predicted'] = mean_core_3_1 * interval_large_jobs_3_1['cpu_req']
print("Accuracy: {}".format(r2_score(interval_large_jobs_3_1['real_pow'], interval_large_jobs_3_1['predicted'])))

quality_large_jobs_3_1 = interval_large_jobs_3_1[interval_large_jobs_3_1['real_pow'] != 0]
quality_large_jobs_3_1 = quality_large_jobs_3_1[quality_large_jobs_3_1['real_pow_quality'] != 0]
print("There are {} jobs completely running on this type of cpu with that quality".format(quality_large_jobs_3_1.shape[0]))
mean_core_3_1 = quality_large_jobs_3_1['real_pow'].sum() / quality_large_jobs_3_1['cpu_req'].sum()
print("The average per core is {} W".format(mean_core_3_1))
quality_large_jobs_3_1['predicted'] = mean_core_3_1 * quality_large_jobs_3_1['cpu_req']
print("Accuracy: {}".format(r2_score(quality_large_jobs_3_1['real_pow'], quality_large_jobs_3_1['predicted'])))

There are 15587 jobs completely running on this type of cpu
The average per core is 22.25328132443491 W
Accuracy: 0.5531975794006317
There are 13793 jobs completely running on this type of cpu with that quality
The average per core is 25.079102763361274 W
Accuracy: 0.6538735694817891


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


### Debug

In [32]:
interval_large_jobs = interval_large_jobs[interval_large_jobs['real_pow_quality'] != 0]
interval_large_jobs.shape

(59542, 19)

In [33]:
interval_large_jobs = interval_large_jobs[interval_large_jobs['real_pow'] != 0]
interval_large_jobs.shape

(57658, 19)

In [35]:
interval_large_jobs_2_1 = interval_large_jobs[interval_large_jobs['node_req'] == interval_large_jobs['n_2_1']]

In [36]:
interval_large_jobs_2_1.shape

(12680, 19)

In [37]:
mean_core_2_1 = interval_large_jobs_2_1['real_pow'].sum() / interval_large_jobs_2_1['cpu_req'].sum()
print(mean_core_2_1)

In [39]:
interval_large_jobs_2_1['predicted'] = mean_core_2_1 * interval_large_jobs_2_1['cpu_req']

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [40]:
#interval_large_jobs_2_1

In [41]:
print("approximation accuracy: {}".format(r2_score(interval_large_jobs_2_1['real_pow'], interval_large_jobs_2_1['predicted'])))

approximation accuracy: 0.6961530048060831


(12327, 20)

In [44]:
interval_large_jobs_2_1_top_quality = interval_large_jobs_2_1[interval_large_jobs_2_1['real_pow_quality'] == 1]
interval_large_jobs_2_1_top_quality.shape
mean_core_2_1_top_quality = interval_large_jobs_2_1['real_pow'].sum() / interval_large_jobs_2_1['cpu_req'].sum()
print(mean_core_2_1_top_quality)
interval_large_jobs_2_1_top_quality['predicted'] = mean_core_2_1_top_quality * interval_large_jobs_2_1_top_quality['cpu_req']
print("approximation accuracy: {}".format(r2_score(interval_large_jobs_2_1_top_quality['real_pow'], interval_large_jobs_2_1_top_quality['predicted'])))

12.188997397054441
approximation accuracy: 0.6984105718416773


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
